# QC-Py-12 - Backtesting et Analyse de Performance

> **Evaluer et analyser les performances de vos strategies de trading**
> Duree: 75 minutes | Niveau: Intermediaire | Python + QuantConnect

---

## Objectifs d'Apprentissage

A la fin de ce notebook, vous serez capable de :

1. Calculer les **metriques de performance** (CAGR, Sharpe, Sortino, Calmar)
2. Analyser les **drawdowns** et periodes de recuperation
3. Evaluer les **statistiques de trades** (win rate, profit factor, expectancy)
4. Comparer une strategie a son **benchmark** (Alpha, Beta)
5. Realiser des **simulations Monte Carlo** pour estimer la robustesse
6. Generer des **rapports de backtest** complets

## Prerequis

- Notebooks QC-Py-01 a QC-Py-08 completes
- Connaissance de pandas et numpy
- Notions de statistiques (moyenne, ecart-type, distributions)

## Structure du Notebook

1. Metriques de Performance (25 min)
2. Analyse des Drawdowns (20 min)
3. Statistiques de Trades (20 min)
4. Comparaison avec Benchmark (15 min)
5. Insights QuantConnect (15 min)
6. Analyse de l'Equity Curve (20 min)
7. Rapport Complet (20 min)

---

## Setup et Imports

In [ ]:
# Imports standards
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional

# Configuration matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Ajouter shared/ au path
import sys
sys.path.append('../shared')

print("Imports reussis")

In [ ]:
# Importer helpers du repository
from backtest_helpers import calculate_metrics, format_backtest_summary, compare_strategies
from plotting import plot_backtest_results, plot_returns_distribution

print("Helpers importes avec succes")

### Generation de Donnees de Backtest Simulees

Pour ce notebook, nous allons generer des donnees de backtest simulees representant une strategie et son benchmark. En environnement reel, ces donnees proviendraient d'un backtest QuantConnect.

In [ ]:
# Generer donnees simulees (2 ans de trading)
np.random.seed(42)
n_days = 504  # ~2 ans de trading
dates = pd.date_range('2022-01-01', periods=n_days, freq='B')  # Business days

# Simuler returns journaliers
# Strategie: mean=0.0005 (12.6% annuel), std=0.015 (23.8% vol)
strategy_returns = np.random.normal(0.0005, 0.015, n_days)

# Benchmark (SPY-like): mean=0.0004 (10% annuel), std=0.012 (19% vol)
benchmark_returns = np.random.normal(0.0004, 0.012, n_days)

# Ajouter correlation entre strategie et benchmark (~0.7)
strategy_returns = 0.6 * strategy_returns + 0.4 * benchmark_returns

# Capital initial
initial_capital = 100000

# Calculer equity curves
strategy_equity = initial_capital * (1 + pd.Series(strategy_returns, index=dates)).cumprod()
benchmark_equity = initial_capital * (1 + pd.Series(benchmark_returns, index=dates)).cumprod()

# Creer DataFrame principal
backtest_df = pd.DataFrame({
    'strategy_equity': strategy_equity,
    'benchmark_equity': benchmark_equity,
    'strategy_returns': strategy_returns,
    'benchmark_returns': benchmark_returns
}, index=dates)

print(f"Donnees generees: {len(backtest_df)} jours de trading")
print(f"Periode: {dates[0].date()} - {dates[-1].date()}")
print(f"\nApercu:")
backtest_df.head()

---

## Partie 1: Metriques de Performance (25 min)

### 1.1 Return Metrics (10 min)

Les metriques de rendement mesurent la performance absolue de la strategie.

#### Total Return et CAGR

- **Total Return**: Gain/perte total en pourcentage
- **CAGR (Compound Annual Growth Rate)**: Taux de croissance annuel compose

$$\text{Total Return} = \frac{\text{Final Value} - \text{Initial Value}}{\text{Initial Value}}$$

$$\text{CAGR} = \left(\frac{\text{Final Value}}{\text{Initial Value}}\right)^{\frac{1}{\text{years}}} - 1$$

In [ ]:
def calculate_return_metrics(equity_series: pd.Series, 
                            start_date: datetime = None,
                            end_date: datetime = None) -> Dict[str, float]:
    """
    Calcule les metriques de rendement de base
    
    Args:
        equity_series: Serie de valeurs du portfolio
        start_date: Date de debut (optionnel)
        end_date: Date de fin (optionnel)
    
    Returns:
        Dict avec total_return, cagr, annualized_return
    """
    if start_date is None:
        start_date = equity_series.index[0]
    if end_date is None:
        end_date = equity_series.index[-1]
    
    initial_value = equity_series.iloc[0]
    final_value = equity_series.iloc[-1]
    
    # Total Return
    total_return = (final_value - initial_value) / initial_value
    
    # CAGR
    years = (end_date - start_date).days / 365.25
    cagr = (final_value / initial_value) ** (1 / years) - 1
    
    # Annualized Return (via daily returns)
    daily_returns = equity_series.pct_change().dropna()
    annualized_return = daily_returns.mean() * 252
    
    return {
        'total_return': total_return,
        'cagr': cagr,
        'annualized_return': annualized_return,
        'years': years
    }

# Appliquer aux donnees
strategy_return_metrics = calculate_return_metrics(backtest_df['strategy_equity'])
benchmark_return_metrics = calculate_return_metrics(backtest_df['benchmark_equity'])

print("=" * 50)
print("METRIQUES DE RENDEMENT")
print("=" * 50)
print(f"\n{'Metrique':<25} {'Strategie':>12} {'Benchmark':>12}")
print("-" * 50)
print(f"{'Total Return':<25} {strategy_return_metrics['total_return']:>11.2%} {benchmark_return_metrics['total_return']:>11.2%}")
print(f"{'CAGR':<25} {strategy_return_metrics['cagr']:>11.2%} {benchmark_return_metrics['cagr']:>11.2%}")
print(f"{'Annualized Return':<25} {strategy_return_metrics['annualized_return']:>11.2%} {benchmark_return_metrics['annualized_return']:>11.2%}")
print(f"{'Periode (annees)':<25} {strategy_return_metrics['years']:>11.2f}")

#### Returns Journaliers, Mensuels et Annuels

In [ ]:
# Returns par periode
strategy_returns_series = backtest_df['strategy_returns']

# Returns mensuels
monthly_returns = (1 + strategy_returns_series).resample('ME').prod() - 1

# Returns annuels
yearly_returns = (1 + strategy_returns_series).resample('YE').prod() - 1

print("Returns Mensuels:")
print(monthly_returns.tail(12))

print("\nReturns Annuels:")
print(yearly_returns)

In [ ]:
# Visualisation: Heatmap des returns mensuels
def plot_monthly_returns_heatmap(returns_series: pd.Series, title: str = "Returns Mensuels"):
    """
    Affiche une heatmap des returns mensuels par annee/mois
    """
    # Calculer returns mensuels
    monthly = (1 + returns_series).resample('ME').prod() - 1
    
    # Creer pivot table
    monthly_df = pd.DataFrame({
        'year': monthly.index.year,
        'month': monthly.index.month,
        'return': monthly.values
    })
    
    pivot = monthly_df.pivot(index='year', columns='month', values='return')
    pivot.columns = ['Jan', 'Fev', 'Mar', 'Avr', 'Mai', 'Jun', 
                     'Jul', 'Aou', 'Sep', 'Oct', 'Nov', 'Dec'][:len(pivot.columns)]
    
    # Plot
    plt.figure(figsize=(14, 4))
    sns.heatmap(pivot * 100, annot=True, fmt='.1f', cmap='RdYlGn', 
                center=0, linewidths=0.5, cbar_kws={'label': 'Return (%)'})
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Mois')
    plt.ylabel('Annee')
    plt.tight_layout()
    plt.show()

plot_monthly_returns_heatmap(strategy_returns_series, "Strategie - Returns Mensuels (%)")

### 1.2 Risk-Adjusted Metrics (15 min)

Les metriques ajustees au risque permettent de comparer des strategies avec des profils de risque differents.

#### Sharpe Ratio

Le **Sharpe Ratio** mesure l'excedent de rendement par unite de risque total.

$$\text{Sharpe Ratio} = \frac{R_p - R_f}{\sigma_p}$$

Ou:
- $R_p$ = Rendement annualise du portfolio
- $R_f$ = Taux sans risque (risk-free rate)
- $\sigma_p$ = Volatilite annualisee du portfolio

**Interpretation**:
- Sharpe < 0 : Mauvais (rendement inferieur au sans risque)
- Sharpe 0-1 : Acceptable
- Sharpe 1-2 : Bon
- Sharpe > 2 : Excellent

In [ ]:
def calculate_sharpe_ratio(returns: pd.Series, 
                          risk_free_rate: float = 0.02,
                          periods_per_year: int = 252) -> float:
    """
    Calcule le Sharpe Ratio annualise
    
    Args:
        returns: Serie de returns journaliers
        risk_free_rate: Taux sans risque annuel (default: 2%)
        periods_per_year: Nombre de periodes par an (252 pour daily)
    
    Returns:
        Sharpe Ratio annualise
    """
    # Annualiser les returns et la volatilite
    annualized_return = returns.mean() * periods_per_year
    annualized_volatility = returns.std() * np.sqrt(periods_per_year)
    
    # Excess return
    excess_return = annualized_return - risk_free_rate
    
    # Sharpe Ratio
    if annualized_volatility == 0:
        return 0.0
    
    sharpe = excess_return / annualized_volatility
    return sharpe

# Calculer Sharpe
sharpe_strategy = calculate_sharpe_ratio(backtest_df['strategy_returns'])
sharpe_benchmark = calculate_sharpe_ratio(backtest_df['benchmark_returns'])

print(f"Sharpe Ratio Strategie:  {sharpe_strategy:.2f}")
print(f"Sharpe Ratio Benchmark:  {sharpe_benchmark:.2f}")

#### Sortino Ratio

Le **Sortino Ratio** est similaire au Sharpe mais ne penalise que la volatilite a la baisse (downside deviation).

$$\text{Sortino Ratio} = \frac{R_p - R_f}{\sigma_d}$$

Ou $\sigma_d$ est la **downside deviation** (ecart-type des returns negatifs uniquement).

**Avantage**: Ne penalise pas la volatilite haussiere (les gros gains).

In [ ]:
def calculate_sortino_ratio(returns: pd.Series,
                           risk_free_rate: float = 0.02,
                           periods_per_year: int = 252) -> float:
    """
    Calcule le Sortino Ratio (downside risk only)
    
    Args:
        returns: Serie de returns journaliers
        risk_free_rate: Taux sans risque annuel
        periods_per_year: Periodes par an
    
    Returns:
        Sortino Ratio annualise
    """
    # Annualized return
    annualized_return = returns.mean() * periods_per_year
    
    # Downside returns only
    downside_returns = returns[returns < 0]
    
    if len(downside_returns) == 0:
        return np.inf  # Pas de pertes
    
    # Downside deviation (annualisee)
    downside_std = downside_returns.std() * np.sqrt(periods_per_year)
    
    if downside_std == 0:
        return 0.0
    
    # Excess return
    excess_return = annualized_return - risk_free_rate
    
    sortino = excess_return / downside_std
    return sortino

# Calculer Sortino
sortino_strategy = calculate_sortino_ratio(backtest_df['strategy_returns'])
sortino_benchmark = calculate_sortino_ratio(backtest_df['benchmark_returns'])

print(f"Sortino Ratio Strategie:  {sortino_strategy:.2f}")
print(f"Sortino Ratio Benchmark:  {sortino_benchmark:.2f}")

#### Calmar Ratio

Le **Calmar Ratio** mesure le rendement par unite de drawdown maximum.

$$\text{Calmar Ratio} = \frac{\text{CAGR}}{|\text{Max Drawdown}|}$$

**Interpretation**: Plus eleve = meilleur. Indique combien de rendement on obtient pour chaque unite de perte maximale potentielle.

In [ ]:
def calculate_max_drawdown(equity_series: pd.Series) -> Tuple[float, pd.Series]:
    """
    Calcule le Max Drawdown et la serie de drawdowns
    
    Returns:
        Tuple (max_drawdown, drawdown_series)
    """
    # Running maximum
    running_max = equity_series.cummax()
    
    # Drawdown series
    drawdown = (equity_series - running_max) / running_max
    
    # Max drawdown
    max_dd = drawdown.min()
    
    return max_dd, drawdown

def calculate_calmar_ratio(equity_series: pd.Series) -> float:
    """
    Calcule le Calmar Ratio
    """
    # CAGR
    metrics = calculate_return_metrics(equity_series)
    cagr = metrics['cagr']
    
    # Max Drawdown
    max_dd, _ = calculate_max_drawdown(equity_series)
    
    if max_dd == 0:
        return np.inf
    
    calmar = cagr / abs(max_dd)
    return calmar

# Calculer Calmar
calmar_strategy = calculate_calmar_ratio(backtest_df['strategy_equity'])
calmar_benchmark = calculate_calmar_ratio(backtest_df['benchmark_equity'])

print(f"Calmar Ratio Strategie:  {calmar_strategy:.2f}")
print(f"Calmar Ratio Benchmark:  {calmar_benchmark:.2f}")

#### Resume des Metriques Ajustees au Risque

In [ ]:
# Volatilite annualisee
vol_strategy = backtest_df['strategy_returns'].std() * np.sqrt(252)
vol_benchmark = backtest_df['benchmark_returns'].std() * np.sqrt(252)

# Max Drawdown
max_dd_strategy, _ = calculate_max_drawdown(backtest_df['strategy_equity'])
max_dd_benchmark, _ = calculate_max_drawdown(backtest_df['benchmark_equity'])

print("=" * 60)
print("METRIQUES AJUSTEES AU RISQUE")
print("=" * 60)
print(f"\n{'Metrique':<25} {'Strategie':>15} {'Benchmark':>15}")
print("-" * 60)
print(f"{'Volatilite Annualisee':<25} {vol_strategy:>14.2%} {vol_benchmark:>14.2%}")
print(f"{'Max Drawdown':<25} {max_dd_strategy:>14.2%} {max_dd_benchmark:>14.2%}")
print(f"{'Sharpe Ratio':<25} {sharpe_strategy:>15.2f} {sharpe_benchmark:>15.2f}")
print(f"{'Sortino Ratio':<25} {sortino_strategy:>15.2f} {sortino_benchmark:>15.2f}")
print(f"{'Calmar Ratio':<25} {calmar_strategy:>15.2f} {calmar_benchmark:>15.2f}")
print("=" * 60)

> **Interpretation**:
> - **Sharpe** > 1 : La strategie genere un bon rendement ajuste au risque total
> - **Sortino** > Sharpe : La volatilite haussiere est superieure a la baissiere (bon signe)
> - **Calmar** > 1 : Le CAGR est superieur au max drawdown (acceptable)

---

## Partie 2: Analyse des Drawdowns (20 min)

### 2.1 Calcul des Drawdowns (10 min)

Un **drawdown** represente la baisse depuis un pic de l'equity curve. C'est une mesure critique du risque.

In [ ]:
def calculate_drawdown_series(equity_curve: pd.Series) -> pd.DataFrame:
    """
    Calcule la serie complete des drawdowns avec statistiques
    
    Returns:
        DataFrame avec equity, running_max, drawdown, drawdown_pct
    """
    df = pd.DataFrame(index=equity_curve.index)
    df['equity'] = equity_curve
    df['running_max'] = equity_curve.cummax()
    df['drawdown'] = equity_curve - df['running_max']
    df['drawdown_pct'] = (equity_curve - df['running_max']) / df['running_max']
    
    return df

# Calculer drawdowns
dd_df = calculate_drawdown_series(backtest_df['strategy_equity'])

print("Statistiques Drawdown:")
print(f"  Max Drawdown:      {dd_df['drawdown_pct'].min():.2%}")
print(f"  Mean Drawdown:     {dd_df['drawdown_pct'].mean():.2%}")
print(f"  Jours en Drawdown: {(dd_df['drawdown_pct'] < 0).sum()} / {len(dd_df)}")

dd_df.tail()

In [ ]:
def calculate_drawdown_duration(drawdown_series: pd.Series) -> Tuple[int, pd.Timestamp, pd.Timestamp]:
    """
    Calcule la duree maximale de drawdown
    
    Returns:
        Tuple (max_duration_days, start_date, end_date)
    """
    # Identifier les periodes de drawdown
    in_drawdown = drawdown_series < 0
    
    # Trouver les debuts et fins de drawdown
    drawdown_starts = in_drawdown & ~in_drawdown.shift(1).fillna(False)
    drawdown_ends = ~in_drawdown & in_drawdown.shift(1).fillna(False)
    
    # Calculer les durees
    max_duration = 0
    max_start = None
    max_end = None
    current_start = None
    
    for date in drawdown_series.index:
        if drawdown_starts.loc[date]:
            current_start = date
        elif drawdown_ends.loc[date] and current_start is not None:
            duration = (date - current_start).days
            if duration > max_duration:
                max_duration = duration
                max_start = current_start
                max_end = date
            current_start = None
    
    # Verifier si on est encore en drawdown a la fin
    if current_start is not None:
        duration = (drawdown_series.index[-1] - current_start).days
        if duration > max_duration:
            max_duration = duration
            max_start = current_start
            max_end = drawdown_series.index[-1]
    
    return max_duration, max_start, max_end

# Calculer duree max
max_dd_duration, dd_start, dd_end = calculate_drawdown_duration(dd_df['drawdown_pct'])

print(f"\nDuree Max Drawdown: {max_dd_duration} jours")
if dd_start and dd_end:
    print(f"Periode: {dd_start.date()} - {dd_end.date()}")

### 2.2 Underwater Plot (10 min)

Le **Underwater Plot** visualise les periodes de drawdown et leur intensite.

In [ ]:
def plot_underwater(drawdown_pct: pd.Series, title: str = "Underwater Plot"):
    """
    Visualise les drawdowns (underwater plot)
    """
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), height_ratios=[2, 1])
    
    # Plot 1: Underwater chart
    ax1 = axes[0]
    ax1.fill_between(drawdown_pct.index, drawdown_pct * 100, 0, 
                     where=drawdown_pct < 0, color='red', alpha=0.3)
    ax1.plot(drawdown_pct.index, drawdown_pct * 100, color='darkred', linewidth=1)
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax1.axhline(y=drawdown_pct.min() * 100, color='red', linestyle='--', 
                label=f'Max DD: {drawdown_pct.min():.2%}')
    ax1.set_ylabel('Drawdown (%)', fontsize=12)
    ax1.set_title(title, fontsize=14, fontweight='bold')
    ax1.legend(loc='lower left')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Distribution des drawdowns
    ax2 = axes[1]
    negative_dd = drawdown_pct[drawdown_pct < 0] * 100
    ax2.hist(negative_dd, bins=30, color='red', alpha=0.6, edgecolor='black')
    ax2.axvline(x=drawdown_pct.min() * 100, color='darkred', linestyle='--', 
                linewidth=2, label=f'Max: {drawdown_pct.min():.2%}')
    ax2.set_xlabel('Drawdown (%)', fontsize=12)
    ax2.set_ylabel('Frequence', fontsize=12)
    ax2.set_title('Distribution des Drawdowns', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_underwater(dd_df['drawdown_pct'], "Strategie - Underwater Plot")

In [ ]:
# Top 5 Drawdowns
def find_top_drawdowns(drawdown_series: pd.Series, n: int = 5) -> pd.DataFrame:
    """
    Identifie les N pires drawdowns
    """
    # Cette implementation simplifiee trouve les N valeurs minimales
    # Une implementation complete identifierait les periodes distinctes
    
    sorted_dd = drawdown_series.sort_values().head(n)
    
    results = []
    for date, dd_val in sorted_dd.items():
        results.append({
            'date': date.strftime('%Y-%m-%d'),
            'drawdown': f"{dd_val:.2%}"
        })
    
    return pd.DataFrame(results)

print("Top 5 pires drawdowns:")
find_top_drawdowns(dd_df['drawdown_pct'], 5)

> **Point Cle**: L'analyse des drawdowns est cruciale pour la gestion du risque. Un max drawdown de 20% signifie qu'a un moment, un investisseur a vu son capital baisser de 20% depuis son pic - psychologiquement difficile a supporter.

---

## Partie 3: Statistiques de Trades (20 min)

### 3.1 Trade Statistics (10 min)

Analysons les statistiques au niveau des trades individuels.

In [ ]:
# Simuler des trades (normalement fournis par le backtest)
np.random.seed(123)
n_trades = 150

# Generer des trades avec win rate ~55% et profit factor ~1.3
trade_outcomes = np.random.choice([1, -1], size=n_trades, p=[0.55, 0.45])

# PnL: gains moyens = 1.5%, pertes moyennes = 1.2%
pnl = np.where(
    trade_outcomes > 0,
    np.random.normal(0.015, 0.008, n_trades),  # Gains
    np.random.normal(-0.012, 0.006, n_trades)   # Pertes
)

# Creer DataFrame de trades
trades_df = pd.DataFrame({
    'trade_id': range(1, n_trades + 1),
    'pnl': pnl,
    'pnl_dollars': pnl * initial_capital,
    'outcome': trade_outcomes
})

print(f"Nombre de trades: {len(trades_df)}")
trades_df.head(10)

In [ ]:
def calculate_trade_statistics(trades: pd.DataFrame) -> Dict[str, float]:
    """
    Calcule les statistiques de trading
    
    Args:
        trades: DataFrame avec colonne 'pnl' (P&L en %)
    
    Returns:
        Dict de statistiques
    """
    # Separer wins et losses
    wins = trades[trades['pnl'] > 0]
    losses = trades[trades['pnl'] < 0]
    
    total_trades = len(trades)
    num_wins = len(wins)
    num_losses = len(losses)
    
    # Win Rate
    win_rate = num_wins / total_trades if total_trades > 0 else 0
    
    # Average Win / Average Loss
    avg_win = wins['pnl'].mean() if len(wins) > 0 else 0
    avg_loss = losses['pnl'].mean() if len(losses) > 0 else 0
    
    # Profit Factor = Sum(Wins) / |Sum(Losses)|
    total_wins = wins['pnl'].sum()
    total_losses = abs(losses['pnl'].sum())
    profit_factor = total_wins / total_losses if total_losses > 0 else np.inf
    
    # Expectancy (Expected value per trade)
    # E = (Win Rate * Avg Win) + ((1 - Win Rate) * Avg Loss)
    expectancy = (win_rate * avg_win) + ((1 - win_rate) * avg_loss)
    
    # Risk/Reward Ratio
    risk_reward = abs(avg_win / avg_loss) if avg_loss != 0 else np.inf
    
    # Largest Win / Loss
    largest_win = wins['pnl'].max() if len(wins) > 0 else 0
    largest_loss = losses['pnl'].min() if len(losses) > 0 else 0
    
    # Consecutive wins/losses
    outcomes = (trades['pnl'] > 0).astype(int)
    # Simplified: just count max consecutive
    max_consec_wins = (outcomes.groupby((outcomes != outcomes.shift()).cumsum())
                       .apply(lambda x: len(x) if x.iloc[0] == 1 else 0).max())
    max_consec_losses = (outcomes.groupby((outcomes != outcomes.shift()).cumsum())
                         .apply(lambda x: len(x) if x.iloc[0] == 0 else 0).max())
    
    return {
        'total_trades': total_trades,
        'winning_trades': num_wins,
        'losing_trades': num_losses,
        'win_rate': win_rate,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'profit_factor': profit_factor,
        'expectancy': expectancy,
        'risk_reward': risk_reward,
        'largest_win': largest_win,
        'largest_loss': largest_loss,
        'max_consec_wins': max_consec_wins,
        'max_consec_losses': max_consec_losses,
        'total_pnl': trades['pnl'].sum()
    }

# Calculer statistiques
trade_stats = calculate_trade_statistics(trades_df)

print("=" * 50)
print("STATISTIQUES DE TRADING")
print("=" * 50)
print(f"\nNombre de trades:         {trade_stats['total_trades']}")
print(f"Trades gagnants:          {trade_stats['winning_trades']}")
print(f"Trades perdants:          {trade_stats['losing_trades']}")
print(f"\nWin Rate:                 {trade_stats['win_rate']:.2%}")
print(f"Average Win:              {trade_stats['avg_win']:.2%}")
print(f"Average Loss:             {trade_stats['avg_loss']:.2%}")
print(f"\nProfit Factor:            {trade_stats['profit_factor']:.2f}")
print(f"Expectancy (per trade):   {trade_stats['expectancy']:.4%}")
print(f"Risk/Reward Ratio:        {trade_stats['risk_reward']:.2f}")
print(f"\nLargest Win:              {trade_stats['largest_win']:.2%}")
print(f"Largest Loss:             {trade_stats['largest_loss']:.2%}")
print(f"\nMax Consec. Wins:         {trade_stats['max_consec_wins']}")
print(f"Max Consec. Losses:       {trade_stats['max_consec_losses']}")
print(f"\nTotal P&L:                {trade_stats['total_pnl']:.2%}")
print("=" * 50)

### 3.2 Trade Distribution (10 min)

Visualisons la distribution des P&L et les patterns temporels.

In [ ]:
# Histogramme des P&L
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Distribution des P&L
ax1 = axes[0]
colors = ['green' if x > 0 else 'red' for x in trades_df['pnl']]
ax1.hist(trades_df['pnl'] * 100, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(x=0, color='black', linestyle='-', linewidth=2)
ax1.axvline(x=trades_df['pnl'].mean() * 100, color='green', linestyle='--', 
            label=f"Mean: {trades_df['pnl'].mean():.2%}")
ax1.axvline(x=trades_df['pnl'].median() * 100, color='orange', linestyle='--',
            label=f"Median: {trades_df['pnl'].median():.2%}")
ax1.set_xlabel('P&L (%)', fontsize=12)
ax1.set_ylabel('Frequence', fontsize=12)
ax1.set_title('Distribution des P&L par Trade', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Cumulative P&L
ax2 = axes[1]
cumulative_pnl = trades_df['pnl'].cumsum()
ax2.plot(trades_df['trade_id'], cumulative_pnl * 100, linewidth=2, color='navy')
ax2.fill_between(trades_df['trade_id'], 0, cumulative_pnl * 100,
                 where=cumulative_pnl >= 0, color='green', alpha=0.3)
ax2.fill_between(trades_df['trade_id'], 0, cumulative_pnl * 100,
                 where=cumulative_pnl < 0, color='red', alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_xlabel('Trade #', fontsize=12)
ax2.set_ylabel('P&L Cumulatif (%)', fontsize=12)
ax2.set_title('P&L Cumulatif par Trade', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Simuler des returns par jour de la semaine
# Ajouter jour de la semaine aux returns
backtest_df['day_of_week'] = backtest_df.index.dayofweek
day_names = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi']

# Returns moyens par jour
returns_by_day = backtest_df.groupby('day_of_week')['strategy_returns'].agg(['mean', 'std', 'count'])
returns_by_day.index = day_names

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Returns par jour de semaine
ax1 = axes[0]
colors = ['green' if x > 0 else 'red' for x in returns_by_day['mean']]
ax1.bar(returns_by_day.index, returns_by_day['mean'] * 100, color=colors, edgecolor='black', alpha=0.7)
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax1.set_ylabel('Return Moyen (%)', fontsize=12)
ax1.set_title('Returns Moyens par Jour de Semaine', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Returns par mois
ax2 = axes[1]
backtest_df['month'] = backtest_df.index.month
returns_by_month = backtest_df.groupby('month')['strategy_returns'].mean()
month_names = ['Jan', 'Fev', 'Mar', 'Avr', 'Mai', 'Jun', 'Jul', 'Aou', 'Sep', 'Oct', 'Nov', 'Dec']
returns_by_month.index = [month_names[i-1] for i in returns_by_month.index]
colors = ['green' if x > 0 else 'red' for x in returns_by_month]
ax2.bar(returns_by_month.index, returns_by_month * 100, color=colors, edgecolor='black', alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_ylabel('Return Moyen (%)', fontsize=12)
ax2.set_title('Returns Moyens par Mois', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Returns par jour de semaine:")
print(returns_by_day)

---

## Partie 4: Comparaison avec Benchmark (15 min)

### 4.1 Alpha et Beta

**Alpha** et **Beta** mesurent la performance relative au benchmark.

- **Beta** : Sensibilite de la strategie aux mouvements du marche
- **Alpha** : Rendement excessif apres ajustement pour le beta

$$\beta = \frac{\text{Cov}(R_s, R_b)}{\text{Var}(R_b)}$$

$$\alpha = R_s - (R_f + \beta \times (R_b - R_f))$$

In [ ]:
def calculate_alpha_beta(strategy_returns: pd.Series,
                        benchmark_returns: pd.Series,
                        risk_free_rate: float = 0.02) -> Tuple[float, float]:
    """
    Calcule Alpha et Beta
    
    Returns:
        Tuple (alpha, beta)
    """
    # Aligner les series
    aligned = pd.DataFrame({
        'strategy': strategy_returns,
        'benchmark': benchmark_returns
    }).dropna()
    
    # Beta = Cov(strategy, benchmark) / Var(benchmark)
    covariance = np.cov(aligned['strategy'], aligned['benchmark'])[0, 1]
    benchmark_variance = aligned['benchmark'].var()
    
    beta = covariance / benchmark_variance if benchmark_variance > 0 else 1.0
    
    # Annualized returns
    strategy_annual = aligned['strategy'].mean() * 252
    benchmark_annual = aligned['benchmark'].mean() * 252
    
    # Alpha = R_strategy - (R_free + Beta * (R_benchmark - R_free))
    alpha = strategy_annual - (risk_free_rate + beta * (benchmark_annual - risk_free_rate))
    
    return alpha, beta

# Calculer Alpha et Beta
alpha, beta = calculate_alpha_beta(
    backtest_df['strategy_returns'],
    backtest_df['benchmark_returns']
)

print("=" * 50)
print("ALPHA & BETA ANALYSIS")
print("=" * 50)
print(f"\nBeta:  {beta:.3f}")
print(f"Alpha: {alpha:.2%} (annualise)")
print("\nInterpretation:")
if beta > 1:
    print(f"  - Beta > 1: Strategie plus volatile que le marche")
elif beta < 1:
    print(f"  - Beta < 1: Strategie moins volatile que le marche")
else:
    print(f"  - Beta = 1: Meme volatilite que le marche")
    
if alpha > 0:
    print(f"  - Alpha > 0: Surperformance ajustee au risque")
else:
    print(f"  - Alpha <= 0: Pas de surperformance ajustee au risque")

In [ ]:
# Visualisation: Scatter plot Strategy vs Benchmark
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Scatter returns
ax1 = axes[0]
ax1.scatter(backtest_df['benchmark_returns'] * 100, 
           backtest_df['strategy_returns'] * 100, 
           alpha=0.5, s=20)

# Ligne de regression
z = np.polyfit(backtest_df['benchmark_returns'], backtest_df['strategy_returns'], 1)
p = np.poly1d(z)
x_line = np.linspace(backtest_df['benchmark_returns'].min(), 
                     backtest_df['benchmark_returns'].max(), 100)
ax1.plot(x_line * 100, p(x_line) * 100, 'r--', linewidth=2, 
         label=f'Beta = {beta:.2f}')

ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax1.set_xlabel('Benchmark Returns (%)', fontsize=12)
ax1.set_ylabel('Strategy Returns (%)', fontsize=12)
ax1.set_title('Strategy vs Benchmark Returns', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Equity curves comparees
ax2 = axes[1]
# Normaliser au meme point de depart
strat_norm = backtest_df['strategy_equity'] / backtest_df['strategy_equity'].iloc[0]
bench_norm = backtest_df['benchmark_equity'] / backtest_df['benchmark_equity'].iloc[0]

ax2.plot(backtest_df.index, strat_norm, label='Strategie', linewidth=2, color='navy')
ax2.plot(backtest_df.index, bench_norm, label='Benchmark', linewidth=2, 
         color='gray', linestyle='--', alpha=0.7)
ax2.axhline(y=1, color='black', linestyle='-', linewidth=0.5)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Valeur Normalisee', fontsize=12)
ax2.set_title('Equity Curves Comparees', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Information Ratio et Tracking Error
def calculate_information_ratio(strategy_returns: pd.Series,
                               benchmark_returns: pd.Series) -> Tuple[float, float]:
    """
    Calcule Information Ratio et Tracking Error
    
    Information Ratio = (R_strategy - R_benchmark) / Tracking Error
    Tracking Error = Std(R_strategy - R_benchmark)
    """
    # Active returns (excess over benchmark)
    active_returns = strategy_returns - benchmark_returns
    
    # Tracking Error (annualise)
    tracking_error = active_returns.std() * np.sqrt(252)
    
    # Mean active return (annualise)
    mean_active = active_returns.mean() * 252
    
    # Information Ratio
    info_ratio = mean_active / tracking_error if tracking_error > 0 else 0
    
    return info_ratio, tracking_error

info_ratio, tracking_error = calculate_information_ratio(
    backtest_df['strategy_returns'],
    backtest_df['benchmark_returns']
)

print(f"\nTracking Error (ann.): {tracking_error:.2%}")
print(f"Information Ratio:     {info_ratio:.2f}")
print("\nInterpretation:")
print("  - IR > 0.5: Bonne generation d'alpha")
print("  - IR > 1.0: Excellente generation d'alpha")

---

## Partie 5: Insights QuantConnect (15 min)

### 5.1 Acceder aux Insights dans QCAlgorithm

Dans un algorithme QuantConnect, vous pouvez acceder aux metriques de backtest via l'API.

In [ ]:
# Code exemple pour QCAlgorithm (a utiliser dans QuantConnect)
qc_example_code = '''
from AlgorithmImports import *

class BacktestAnalysisAlgorithm(QCAlgorithm):
    """
    Exemple d'acces aux statistiques de backtest dans QuantConnect
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Tracking des trades
        self.trade_count = 0
        self.winning_trades = 0
        self.losing_trades = 0
    
    def OnData(self, data):
        if not self.Portfolio.Invested:
            self.SetHoldings(self.symbol, 1.0)
    
    def OnOrderEvent(self, orderEvent):
        """Tracker les trades"""
        if orderEvent.Status == OrderStatus.Filled:
            self.trade_count += 1
    
    def OnEndOfAlgorithm(self):
        """
        Appele a la fin du backtest - acces aux statistiques
        """
        # Statistiques de base
        self.Debug("=" * 50)
        self.Debug("BACKTEST COMPLETE")
        self.Debug("=" * 50)
        
        # Portfolio stats
        self.Debug(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:,.2f}")
        self.Debug(f"Total Trades: {self.trade_count}")
        
        # Acces aux trades fermes via TradeBuilder
        closed_trades = self.TradeBuilder.ClosedTrades
        
        if closed_trades:
            total_pnl = sum(t.ProfitLoss for t in closed_trades)
            wins = [t for t in closed_trades if t.ProfitLoss > 0]
            losses = [t for t in closed_trades if t.ProfitLoss < 0]
            
            self.Debug(f"Closed Trades: {len(closed_trades)}")
            self.Debug(f"Total P&L: ${total_pnl:,.2f}")
            self.Debug(f"Win Rate: {len(wins)/len(closed_trades):.2%}")
            
            # Details par trade
            for trade in closed_trades[-5:]:  # Derniers 5 trades
                self.Debug(f"  {trade.Symbol}: Entry={trade.EntryTime}, "
                          f"Exit={trade.ExitTime}, PnL=${trade.ProfitLoss:,.2f}")
        
        # Acces aux Insights (si utilises)
        insights = self.Insights.GetInsights()
        if insights:
            self.Debug(f"Total Insights Generated: {len(insights)}")
            
            # Analyser les insights
            correct_insights = [i for i in insights if i.EstimatedValue > 0]
            self.Debug(f"Correct Insights: {len(correct_insights)}/{len(insights)}")
'''

print("Exemple de code QCAlgorithm pour acceder aux statistiques:")
print(qc_example_code)

### 5.2 Structure des Resultats de Backtest QuantConnect

QuantConnect fournit des statistiques detaillees apres chaque backtest.

In [ ]:
# Simuler les statistiques type QuantConnect
qc_backtest_stats = {
    # Performance
    'Total Return': f"{strategy_return_metrics['total_return']:.2%}",
    'CAGR': f"{strategy_return_metrics['cagr']:.2%}",
    'Sharpe Ratio': f"{sharpe_strategy:.2f}",
    'Sortino Ratio': f"{sortino_strategy:.2f}",
    
    # Risk
    'Annual Volatility': f"{vol_strategy:.2%}",
    'Max Drawdown': f"{max_dd_strategy:.2%}",
    'Calmar Ratio': f"{calmar_strategy:.2f}",
    
    # Alpha/Beta
    'Alpha': f"{alpha:.2%}",
    'Beta': f"{beta:.2f}",
    'Information Ratio': f"{info_ratio:.2f}",
    'Tracking Error': f"{tracking_error:.2%}",
    
    # Trading
    'Total Trades': trade_stats['total_trades'],
    'Win Rate': f"{trade_stats['win_rate']:.2%}",
    'Profit Factor': f"{trade_stats['profit_factor']:.2f}",
    'Average Win': f"{trade_stats['avg_win']:.2%}",
    'Average Loss': f"{trade_stats['avg_loss']:.2%}",
}

# Afficher comme tableau QuantConnect
print("=" * 60)
print("QUANTCONNECT BACKTEST SUMMARY")
print("=" * 60)
print(f"\n{'Performance Metrics':<30}")
print("-" * 40)
for key in ['Total Return', 'CAGR', 'Sharpe Ratio', 'Sortino Ratio']:
    print(f"  {key:<25} {qc_backtest_stats[key]:>12}")

print(f"\n{'Risk Metrics':<30}")
print("-" * 40)
for key in ['Annual Volatility', 'Max Drawdown', 'Calmar Ratio']:
    print(f"  {key:<25} {qc_backtest_stats[key]:>12}")

print(f"\n{'Alpha/Beta Analysis':<30}")
print("-" * 40)
for key in ['Alpha', 'Beta', 'Information Ratio', 'Tracking Error']:
    print(f"  {key:<25} {qc_backtest_stats[key]:>12}")

print(f"\n{'Trading Statistics':<30}")
print("-" * 40)
for key in ['Total Trades', 'Win Rate', 'Profit Factor', 'Average Win', 'Average Loss']:
    print(f"  {key:<25} {str(qc_backtest_stats[key]):>12}")

print("=" * 60)

---

## Partie 6: Analyse de l'Equity Curve (20 min)

### 6.1 Simulation Monte Carlo (10 min)

La **simulation Monte Carlo** permet d'estimer la distribution des resultats possibles en reshufflant les trades.

In [ ]:
def monte_carlo_simulation(returns: pd.Series, 
                          n_simulations: int = 1000,
                          initial_capital: float = 100000) -> Dict:
    """
    Realise une simulation Monte Carlo sur les returns
    
    Args:
        returns: Serie de returns journaliers
        n_simulations: Nombre de simulations
        initial_capital: Capital initial
    
    Returns:
        Dict avec statistiques et equity curves simulees
    """
    np.random.seed(42)
    
    returns_array = returns.values
    n_periods = len(returns_array)
    
    final_values = []
    max_drawdowns = []
    equity_curves = []
    
    for i in range(n_simulations):
        # Shuffle les returns (bootstrap avec remplacement)
        shuffled_returns = np.random.choice(returns_array, size=n_periods, replace=True)
        
        # Calculer equity curve
        equity = initial_capital * np.cumprod(1 + shuffled_returns)
        
        # Final value
        final_values.append(equity[-1])
        
        # Max drawdown
        running_max = np.maximum.accumulate(equity)
        drawdown = (equity - running_max) / running_max
        max_drawdowns.append(drawdown.min())
        
        # Stocker quelques curves pour visualisation
        if i < 100:  # Garder 100 curves
            equity_curves.append(equity)
    
    # Percentiles
    percentiles = [5, 25, 50, 75, 95]
    value_percentiles = np.percentile(final_values, percentiles)
    dd_percentiles = np.percentile(max_drawdowns, percentiles)
    
    return {
        'final_values': final_values,
        'max_drawdowns': max_drawdowns,
        'equity_curves': equity_curves,
        'value_percentiles': dict(zip(percentiles, value_percentiles)),
        'dd_percentiles': dict(zip(percentiles, dd_percentiles)),
        'mean_final': np.mean(final_values),
        'std_final': np.std(final_values)
    }

# Executer simulation
print("Execution de 1000 simulations Monte Carlo...")
mc_results = monte_carlo_simulation(backtest_df['strategy_returns'], n_simulations=1000)

print("\nResultats Monte Carlo:")
print(f"\nValeur Finale (distribution):")
print(f"  5e percentile:   ${mc_results['value_percentiles'][5]:,.0f}")
print(f"  25e percentile:  ${mc_results['value_percentiles'][25]:,.0f}")
print(f"  Mediane (50e):   ${mc_results['value_percentiles'][50]:,.0f}")
print(f"  75e percentile:  ${mc_results['value_percentiles'][75]:,.0f}")
print(f"  95e percentile:  ${mc_results['value_percentiles'][95]:,.0f}")

print(f"\nMax Drawdown (distribution):")
print(f"  5e percentile:   {mc_results['dd_percentiles'][5]:.2%}")
print(f"  Mediane (50e):   {mc_results['dd_percentiles'][50]:.2%}")
print(f"  95e percentile:  {mc_results['dd_percentiles'][95]:.2%}")

In [ ]:
# Visualisation Monte Carlo
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Equity curves simulees
ax1 = axes[0]
for curve in mc_results['equity_curves'][:50]:  # 50 premieres
    ax1.plot(curve / 1000, alpha=0.1, color='blue')

# Equity curve reelle
ax1.plot(backtest_df['strategy_equity'].values / 1000, 
         color='red', linewidth=2, label='Realise')
ax1.set_xlabel('Jours', fontsize=12)
ax1.set_ylabel('Valeur (k$)', fontsize=12)
ax1.set_title('Simulations Monte Carlo', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Distribution des valeurs finales
ax2 = axes[1]
ax2.hist(np.array(mc_results['final_values']) / 1000, bins=50, 
         color='steelblue', edgecolor='black', alpha=0.7)
ax2.axvline(x=backtest_df['strategy_equity'].iloc[-1] / 1000, 
            color='red', linestyle='--', linewidth=2, label='Realise')
ax2.axvline(x=mc_results['value_percentiles'][50] / 1000,
            color='green', linestyle='--', linewidth=2, label='Mediane MC')
ax2.set_xlabel('Valeur Finale (k$)', fontsize=12)
ax2.set_ylabel('Frequence', fontsize=12)
ax2.set_title('Distribution Valeurs Finales', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Distribution des max drawdowns
ax3 = axes[2]
ax3.hist(np.array(mc_results['max_drawdowns']) * 100, bins=50,
         color='red', edgecolor='black', alpha=0.5)
ax3.axvline(x=max_dd_strategy * 100, color='darkred', linestyle='--',
            linewidth=2, label='Realise')
ax3.set_xlabel('Max Drawdown (%)', fontsize=12)
ax3.set_ylabel('Frequence', fontsize=12)
ax3.set_title('Distribution Max Drawdowns', fontsize=14, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6.2 Rolling Statistics (10 min)

Les statistiques glissantes permettent d'identifier les changements de regime.

In [ ]:
def calculate_rolling_statistics(returns: pd.Series, 
                                window: int = 60) -> pd.DataFrame:
    """
    Calcule les statistiques rolling
    
    Args:
        returns: Serie de returns
        window: Fenetre (default: 60 jours)
    
    Returns:
        DataFrame avec rolling stats
    """
    df = pd.DataFrame(index=returns.index)
    
    # Rolling Sharpe (annualise)
    rolling_mean = returns.rolling(window).mean() * 252
    rolling_std = returns.rolling(window).std() * np.sqrt(252)
    df['rolling_sharpe'] = (rolling_mean - 0.02) / rolling_std  # Rf = 2%
    
    # Rolling Volatility
    df['rolling_volatility'] = rolling_std
    
    # Rolling Return (annualise)
    df['rolling_return'] = rolling_mean
    
    # Rolling Win Rate
    df['rolling_winrate'] = (returns > 0).rolling(window).mean()
    
    return df

# Calculer rolling stats
rolling_df = calculate_rolling_statistics(backtest_df['strategy_returns'], window=60)

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Rolling Sharpe
ax1 = axes[0, 0]
ax1.plot(rolling_df.index, rolling_df['rolling_sharpe'], linewidth=2, color='navy')
ax1.axhline(y=0, color='red', linestyle='--', linewidth=1)
ax1.axhline(y=1, color='green', linestyle='--', linewidth=1, alpha=0.5)
ax1.fill_between(rolling_df.index, rolling_df['rolling_sharpe'], 0,
                 where=rolling_df['rolling_sharpe'] > 0, color='green', alpha=0.2)
ax1.fill_between(rolling_df.index, rolling_df['rolling_sharpe'], 0,
                 where=rolling_df['rolling_sharpe'] < 0, color='red', alpha=0.2)
ax1.set_ylabel('Sharpe Ratio', fontsize=12)
ax1.set_title('Rolling Sharpe Ratio (60j)', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Rolling Volatility
ax2 = axes[0, 1]
ax2.plot(rolling_df.index, rolling_df['rolling_volatility'] * 100, 
         linewidth=2, color='darkorange')
ax2.axhline(y=rolling_df['rolling_volatility'].mean() * 100, color='red', 
            linestyle='--', label=f"Mean: {rolling_df['rolling_volatility'].mean():.2%}")
ax2.set_ylabel('Volatilite (%)', fontsize=12)
ax2.set_title('Rolling Volatility (60j)', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Rolling Return
ax3 = axes[1, 0]
ax3.plot(rolling_df.index, rolling_df['rolling_return'] * 100, 
         linewidth=2, color='darkgreen')
ax3.axhline(y=0, color='red', linestyle='--', linewidth=1)
ax3.fill_between(rolling_df.index, rolling_df['rolling_return'] * 100, 0,
                 where=rolling_df['rolling_return'] > 0, color='green', alpha=0.2)
ax3.fill_between(rolling_df.index, rolling_df['rolling_return'] * 100, 0,
                 where=rolling_df['rolling_return'] < 0, color='red', alpha=0.2)
ax3.set_ylabel('Return Annualise (%)', fontsize=12)
ax3.set_title('Rolling Return (60j)', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

# Rolling Win Rate
ax4 = axes[1, 1]
ax4.plot(rolling_df.index, rolling_df['rolling_winrate'] * 100, 
         linewidth=2, color='purple')
ax4.axhline(y=50, color='red', linestyle='--', linewidth=1, label='50%')
ax4.set_ylabel('Win Rate (%)', fontsize=12)
ax4.set_title('Rolling Win Rate (60j)', fontsize=14, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

for ax in axes.flat:
    ax.set_xlabel('Date', fontsize=12)

plt.tight_layout()
plt.show()

> **Interpretation**:
> - **Rolling Sharpe**: Identifie les periodes de bonne/mauvaise performance ajustee au risque
> - **Rolling Volatility**: Detecte les changements de regime de volatilite
> - **Rolling Win Rate < 50%**: Periodes difficiles necessitant peut-etre une adaptation

---

## Partie 7: Rapport Complet (20 min)

### 7.1 Generer un Rapport de Backtest Complet

In [ ]:
def generate_backtest_report(equity_series: pd.Series,
                            benchmark_series: pd.Series,
                            trades_df: pd.DataFrame,
                            strategy_name: str = "Strategie") -> str:
    """
    Genere un rapport de backtest complet en texte
    """
    # Calculer toutes les metriques
    returns = equity_series.pct_change().dropna()
    bench_returns = benchmark_series.pct_change().dropna()
    
    # Return metrics
    ret_metrics = calculate_return_metrics(equity_series)
    
    # Risk metrics
    sharpe = calculate_sharpe_ratio(returns)
    sortino = calculate_sortino_ratio(returns)
    calmar = calculate_calmar_ratio(equity_series)
    max_dd, _ = calculate_max_drawdown(equity_series)
    vol = returns.std() * np.sqrt(252)
    
    # Alpha/Beta
    alpha, beta = calculate_alpha_beta(returns, bench_returns)
    info_ratio, track_error = calculate_information_ratio(returns, bench_returns)
    
    # Trade stats
    trade_stats = calculate_trade_statistics(trades_df)
    
    # Generer le rapport
    report = f"""
{'='*70}
RAPPORT DE BACKTEST - {strategy_name.upper()}
{'='*70}

Periode: {equity_series.index[0].strftime('%Y-%m-%d')} - {equity_series.index[-1].strftime('%Y-%m-%d')}
Duree: {ret_metrics['years']:.1f} annees ({len(equity_series)} jours de trading)
Capital Initial: ${equity_series.iloc[0]:,.0f}
Capital Final: ${equity_series.iloc[-1]:,.0f}

{'-'*70}
METRIQUES DE PERFORMANCE
{'-'*70}
  Total Return:           {ret_metrics['total_return']:>10.2%}
  CAGR:                   {ret_metrics['cagr']:>10.2%}
  Annualized Return:      {ret_metrics['annualized_return']:>10.2%}

{'-'*70}
METRIQUES DE RISQUE
{'-'*70}
  Volatilite (ann.):      {vol:>10.2%}
  Max Drawdown:           {max_dd:>10.2%}
  Sharpe Ratio:           {sharpe:>10.2f}
  Sortino Ratio:          {sortino:>10.2f}
  Calmar Ratio:           {calmar:>10.2f}

{'-'*70}
COMPARAISON BENCHMARK
{'-'*70}
  Alpha (ann.):           {alpha:>10.2%}
  Beta:                   {beta:>10.2f}
  Information Ratio:      {info_ratio:>10.2f}
  Tracking Error:         {track_error:>10.2%}

{'-'*70}
STATISTIQUES DE TRADING
{'-'*70}
  Nombre de Trades:       {trade_stats['total_trades']:>10}
  Win Rate:               {trade_stats['win_rate']:>10.2%}
  Profit Factor:          {trade_stats['profit_factor']:>10.2f}
  Expectancy:             {trade_stats['expectancy']:>10.4%}
  Average Win:            {trade_stats['avg_win']:>10.2%}
  Average Loss:           {trade_stats['avg_loss']:>10.2%}
  Risk/Reward:            {trade_stats['risk_reward']:>10.2f}
  Max Consec. Wins:       {trade_stats['max_consec_wins']:>10}
  Max Consec. Losses:     {trade_stats['max_consec_losses']:>10}

{'='*70}
FIN DU RAPPORT
{'='*70}
"""
    return report

# Generer et afficher le rapport
report = generate_backtest_report(
    backtest_df['strategy_equity'],
    backtest_df['benchmark_equity'],
    trades_df,
    "MA Crossover Strategy"
)

print(report)

In [ ]:
# Utiliser les helpers du repository
# Calculer via helper standardise
metrics_helper = calculate_metrics(
    backtest_df['strategy_equity'],
    benchmark=backtest_df['benchmark_equity']
)

# Formater via helper
summary_helper = format_backtest_summary(metrics_helper, "MA Crossover (via helper)")
print(summary_helper)

In [ ]:
# Visualisation complete avec helper
# Preparer DataFrame pour plot_backtest_results
results_for_plot = pd.DataFrame({
    'equity': backtest_df['strategy_equity'],
    'daily_returns': backtest_df['strategy_returns'],
    'drawdown': dd_df['drawdown_pct']
}, index=backtest_df.index)

# Plot
plot_backtest_results(
    results_for_plot,
    benchmark=backtest_df['benchmark_equity'],
    title='MA Crossover Strategy - Backtest Results'
)

In [ ]:
# Distribution des returns avec helper
plot_returns_distribution(
    backtest_df['strategy_returns'],
    title='MA Crossover - Returns Distribution'
)

In [ ]:
# Comparaison de strategies via helper
strategies = {
    'MA Crossover': backtest_df['strategy_equity'],
    'Benchmark (SPY)': backtest_df['benchmark_equity']
}

comparison = compare_strategies(strategies)

print("\nComparaison des Strategies:")
print(comparison.round(4))

---

## Conclusion et Prochaines Etapes

### Recapitulatif

Dans ce notebook, nous avons appris a:

1. **Calculer les metriques de performance**: Total Return, CAGR, rendements par periode
2. **Evaluer le risque ajuste**: Sharpe, Sortino, Calmar ratios
3. **Analyser les drawdowns**: Max drawdown, duree, underwater plot
4. **Statistiques de trades**: Win rate, profit factor, expectancy, risk/reward
5. **Comparer au benchmark**: Alpha, Beta, Information Ratio, Tracking Error
6. **Simulations Monte Carlo**: Distribution des resultats possibles
7. **Rolling statistics**: Detection de changements de regime
8. **Generer des rapports complets**: Via fonctions custom et helpers

### Points Cles a Retenir

| Metrique | Ce qu'elle mesure | Bon seuil |
|----------|-------------------|----------|
| **Sharpe Ratio** | Return / Risque total | > 1.0 |
| **Sortino Ratio** | Return / Risque baissier | > 1.5 |
| **Calmar Ratio** | CAGR / Max Drawdown | > 1.0 |
| **Win Rate** | % trades gagnants | > 50% |
| **Profit Factor** | Gains / Pertes | > 1.5 |
| **Alpha** | Surperformance ajustee | > 0% |
| **Information Ratio** | Alpha / Tracking Error | > 0.5 |

### Limitations et Precautions

1. **Overfitting**: Des metriques excellentes sur backtest peuvent ne pas se reproduire en live
2. **Survivorship Bias**: S'assurer que les donnees incluent les titres delisttes
3. **Lookahead Bias**: Verifier que les signaux n'utilisent pas d'information future
4. **Transaction Costs**: Inclure les frais et slippage realistes
5. **Market Impact**: Les gros ordres peuvent impacter les prix

### Ressources

- [QuantConnect Backtest Results](https://www.quantconnect.com/docs/v2/our-platform/backtesting/results)
- Helpers: `shared/backtest_helpers.py`, `shared/plotting.py`
- Notebook suivant: **QC-Py-13** (selon curriculum)

---

**Notebook complete. Les metriques de backtest sont essentielles pour evaluer objectivement vos strategies.**